# **Stacking from scratch**

Berikan penjelasan terkait Stacking!

Source : https://www.geeksforgeeks.org/machine-learning/stacking-in-machine-learning/
Youtube : https://www.youtube.com/watch?v=a4IS1Ai7GCI

In [1]:
# Import Libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

**Pendefinisian class stack**

Bertujuan untuk membuat kerangka kerja ensemble learning berbasis stacking dan blending secara modular dan fleksibel. Class ini memungkinkan pengguna untuk menggabungkan beberapa model dasar (base learners) dan satu model meta (final estimator) untuk menghasilkan prediksi akhir yang lebih akurat. Dengan mengatur parameter seperti metode cross-validation, penggunaan blending, serta paralelisasi proses, class ini memberikan kontrol penuh kepada pengguna dalam menerapkan teknik stacking sesuai kebutuhan. Kegunaan utamanya adalah sebagai alat bantu untuk mengimplementasikan ensemble model dari nol (from scratch) tanpa tergantung pada fungsi otomatis dari pustaka seperti scikit-learn, sehingga cocok untuk pembelajaran konsep maupun eksperimen lanjutan dalam machine learning.

Buat class bernama Stack yang berisi <br>

Attribute:


Method:


In [2]:
# Definisikan class stack
class Stack:
    def __init__(self, base_models, meta_model):
        self.base_models = base_models
        self.meta_model = meta_model

    def fit(self, X, y):
        # Simpan prediksi base model
        meta_features = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            model.fit(X, y)
            meta_features[:, i] = model.predict(X)
        # Latih meta model
        self.meta_model.fit(meta_features, y)

    def predict(self, X):
        meta_features = np.column_stack([
            model.predict(X) for model in self.base_models
        ])
        return self.meta_model.predict(meta_features)

### **Upload datasets**

In [3]:
# Load dataset iris dari sklearn
from sklearn.datasets import load_iris
iris = load_iris(as_frame=True)
glass_df = iris.frame

# Pisahkan fitur dan target
X1 = glass_df.drop(columns=["target"])
y1 = glass_df["target"]

# Encode target
le = LabelEncoder()
y1 = le.fit_transform(y1)

# Split data
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=0)

# **Model training and evaluation of the obtained results**
Both in the case of classification and regression, stacking and blending showed the same and not the best results. As a rule, this situation occurs for two reasons: given that metadata is based on predictions of basic models, the presence of weak basic models can reduce the accuracy of stronger ones which will reduce the final prediction as a whole. Also a small amount of training data often leads to overfitting which in turn reduces the accuracy of predictions.

In this case the problem can be partially solved by setting stack_method='predict_proba' when each basic classifier outputs class membership probabilities instead of the classes themselves which can help increase accuracy in the case of non-mutually exclusive classes. Also this method works better with noise in the data. As you can see this method has significantly increased the accuracy of the model. With the right selection of models and hyperparameters the accuracy will be even higher.

Most often stacking shows slightly better results than blending due to the use of k-fold cross-validation but usually the difference is noticeable only on a large amount of data.

In [4]:
# StackingClassifier dan Blending Classifier
from sklearn.ensemble import StackingClassifier

estimators = [
    ('rf', RandomForestClassifier(n_estimators=50, random_state=0)),
    ('gb', GradientBoostingClassifier(random_state=0))
]

stack_model = StackingClassifier(
    estimators=estimators,
    final_estimator=LogisticRegression(),
    stack_method='predict_proba'
)

stack_model.fit(X1_train, y1_train)
y_pred_stack = stack_model.predict(X1_test)
print("Akurasi Stacking:", accuracy_score(y1_test, y_pred_stack))

Akurasi Stacking: 1.0


**StackingClassifier (scikit-learn)**


* Merupakan implementasi resmi stacking untuk klasifikasi dalam scikit-learn.

* Mempermudah proses ensemble dengan base learners dan final estimator (meta learner) dalam satu objek.

* Sudah menangani cross-validation secara internal sehingga mengurangi risiko data leakage.

* Mendukung parameter passthrough=True jika ingin menggabungkan fitur asli dengan meta-features.

* Cocok untuk digunakan dalam pipeline dan produksi karena stabil dan teruji.

In [6]:
# StackingClassifier (scikit-learn)
sk_estimators = [
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('gb', GradientBoostingClassifier(random_state=42))
]
stack_methods = ['predict', 'predict_proba']

# loop untuk membandingkan metode stacking
for i, method in enumerate(stack_methods):
    print(f"\n=== Metode: {method} ===")

    # buat model stacking
    stack_model = StackingClassifier(
        estimators=sk_estimators,
        final_estimator=LogisticRegression(),
        stack_method=method
    )

    # latih model
    stack_model.fit(X1_train, y1_train)

    # prediksi
    y_pred = stack_model.predict(X1_test)

    # evaluasi
    acc = accuracy_score(y1_test, y_pred)
    print(f"Akurasi: {acc:.4f}")


=== Metode: predict ===
Akurasi: 1.0000

=== Metode: predict_proba ===
Akurasi: 1.0000
